In [3]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [4]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)
path

Path('/home/kyan/.fastai/archive/bluebook-for-bulldozers')

In [6]:
from kaggle import api

if not path.exists():
    path.mkdir(parents=true)
    api.competition_download_cli(comp, path=path)
    shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

path.ls(file_type='text')

ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Sun, 25 Feb 2024 14:27:28 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=beb5d7b700d19fb7e8141fc813236e15; max-age=2626560; path=/, .AspNetCore.Mvc.CookieTempDataProvider=CfDJ8Fax40yAg55Jn-jR9HR78eHEJtI3D4mBcxIBImAf5X7qi_OnQSu3leRgoEgy3AVsh4LilTHbz83-bY7rXJcatUlrm0qkFbgLjtvXJQw2wRMsQyDGLarm6URTB73RQGJiVS7jlP7NHsqqT93CwWB2X16YV1n_h9GLkcwFOr5EQMCgfYBrkFehm8RUOayqzMHdJEQHZS7ulQPe8vdNo46A39_T49O-b8qHzZYO0MlHT4tBftz41PbAjJOETcLk0QXsjY2YeXE4uVEL1g; path=/; samesite=lax; httponly, GCLB=CMeooKnXw7izigE; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/competitions/data/download-all/bluebook-for-bulldozers', 'X-Kaggle-MillisecondsElapsed': '19', 'X-Kaggle-RequestId': '6f5b9b04770a45c97294c5afe17eb905', 'X-Kaggle-ApiVersion': '1.6.7', 'X-Kaggle-HubVersion': '0